# Hades: Practical Decentralized Identity with Full Accountability and Fine-grained Sybil-resistance from ACSAC 2023 from ACSAC 2023

## Install necessary libraries

In [27]:
!pip install --upgrade mergetb
!pip install timeout-timer

## Import necessary libraries

In [28]:
import time, socket
from timeout_timer import timeout, TimeoutInterrupt
import subprocess

from mergetb.workspace import User, Experiment, Project
from mergetb.realize import Realization
from mergetb.materialize import Materialization
from mergetb.grpc_client import MergeGRPCError
from mergetb.types import StatusType

## Log in as user¶

In [33]:
username='marine'
passw='0m@rine1!!'

try:
    u = User(username, passw)
    u.login()
except MergeGRPCError as e:
    print(f"Error logging in: {e}")
else:
    print("Logged in as user", u.username)

Logged in as user marine


## Find or create experiment

In [34]:
try:
    # load the user's personal project
    p = Project(username)
    resp = p.get()

    # look for the experiment, create if it does not exist
    for exp in resp.project.experiments:
        if exp == 'hadestry':
            print('experiment hadestry already exists')
            break
    else:
        e = Experiment('hadestry', username)
        e.create()
        print('experiment hadestry created')
    
except MergeGRPCError as e:
    print(e)
#proj = m.find_project(username)
#exp = m.find_experiment('hades', proj.name)
#if exp == None:
#    exp = m.create_experiment('hades', proj.name)

experiment hadestry created


## Push model, allocate resources and attach the experiment to the XDC

In [35]:
RLZ_TIMEOUT=15
MTZ_TIMEOUT=60

try:
    exp = Experiment('hadestry', username)
    rev = exp.push_model('/project/marine/notebooks/hades.model').revision
    print("Pushed experiment revision", rev)
    time.sleep(5)
    
    # reserve resources through a lease named 'v1' (name is arbitrary)
    exp.reserve('vtry', revision=rev)

    # wait up for 10 sec for reservation to succeed
    try:
        rlz = Realization('vtry', exp.name, exp.project)
        with timeout(RLZ_TIMEOUT):
            while True:
                resp = rlz.get()
                status = resp.status.highest_status

                if status == StatusType.Success:
                    print('leased resources')
                    break
    except TimeoutInterrupt:
        raise Exception('reservation did not succeed within %d seconds' % RLZ_TIMEOUT)

    # activate experiment on this lease
    exp.activate('vtry')

    # wait up to 60 sec for activation to succeed
    try:
        mtz = Materialization('vtry', exp.name, exp.project)
        with timeout(MTZ_TIMEOUT):
            while True:
                resp = mtz.get()
                status = resp.status.highest_status

                if status == StatusType.Success:
                    print('started experiment')
                    break
    except TimeoutInterrupt:
        raise Exception('activation did not succeed within %d seconds' % MTZ_TIMEOUT)

    hostname = socket.gethostname().split('-')[0].split('.')[0]
    exp.attach_xdc('vtry', hostname, username)
    time.sleep(15)
    print("Attached experiment is now running")
except MergeGRPCError as e:
    print(e)

Pushed experiment revision aefaee38b4386494b607a2f1c7f190763ff37d25
leased resources
  [stream_terminated: attempting try 2 out of 5]
started experiment
Attached experiment is now running


## Run commands in experiment to clone the artifact, install prerequisites, fix some bugs and execute one script in the artifact¶

In [38]:
exp = Experiment('hades', username)
def print_lines(label, lines, elines):
    print(f"--- {label} ---")
    for line in lines:
        print(line.strip())
    for eline in elines:
        print(eline.strip())

installpackages = "sudo apt-get update && sudo apt install -y git"
clonerepo = "git clone https://github.com/didnet/Hades.git"
packagesartifact1 = (
    "sudo apt update; sudo apt install -y build-essential;"
    "curl --proto '=https' --tlsv1.2 https://sh.rustup.rs -sSf | sh -s -- -y;"
    ". $HOME/.cargo/env"
)
packagesartifact2 = ". $HOME/.cargo/env; git clone https://github.com/iden3/circom.git; cd circom; cargo build --release; cargo install --path circom; cd .."
packagesartifact3 = ". $HOME/.cargo/env; cd Hades; circom --r1cs --wasm circuits/pseudonym_check.circom; circom --r1cs --wasm circuits/sybil_check.circom; circom --r1cs --wasm circuits/pedersen_commit.circom; circom --r1cs --wasm circuits/tpke_single.circom; cargo build --release"
letwallet =""". $HOME/.cargo/env; cd Hades/src; sed -i '29s|let wallet = ".*"|let wallet = "Your_new_number_here";|' /home/marine/Hades/src/bench.rs"""
walletid=""" sed -i '33s|let client = SignerMiddleware::new(provider.clone(), wallet.with_chain_id(97u64));|let client = SignerMiddleware::new(provider.clone(), wallet.with_chain_id(new_wallet_id));|' /home/marine/Hades/src/bench.rs;  cd .."""

runcode = ". $HOME/.cargo/env ; Hades/target/release/hades"

# Install packages
#print("Installing packages, this may take a while...")
#lines, elines = exp.exec_on_node(username, "nodehades", installpackages)
#print_lines("Installing packages", lines, elines)

# Clone artifact repository
#print("Cloning artifact repository")
#lines, elines = exp.exec_on_node(username, "nodehades", clonerepo)
#print_lines("Cloning artifact repository", lines, elines)

# Install additional packages and configure environment for Rust
#print("Installing some more packages")
#lines, elines = exp.exec_on_node(username, "nodehades", packagesartifact1)
#print_lines("Installing more packages", lines, elines)

# Execute further installation steps
#print("Executing further installation steps")
#lines, elines = exp.exec_on_node(username, "nodehades", packagesartifact2)
#print_lines("Installing artifacts", lines, elines)
#lines, elines = exp.exec_on_node(username, "nodehades", packagesartifact3)
#print_lines("Installing artifacts", lines, elines)

#modify wallet id
print("Modification of wallet id, line 29 of bench.rs, you may need to modify line 33 as well")
lines, elines = exp.exec_on_node(username, "nodehades", letwallet)
lines, elines = exp.exec_on_node(username, "nodehades", walletid)


# Execute the final code for utility comparison
print("Will now execute the code for theoretical utility comparison")
lines, elines = exp.exec_on_node(username, "nodehades", runcode)
print_lines("Running comparison code", lines, elines)


Modification of wallet id, line 29 of bench.rs, you may need to modify line 33 as well
Will now execute the code for theoretical utility comparison
--- Running comparison code ---
1. Start setting up the committee:
thread 'main' panicked at src/main.rs:6:30:
called `Result::unwrap()` on an `Err` value: No such file or directory (os error 2)

Location:
/home/marine/Hades/src/bench.rs:38:19
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
